In [10]:
import pandas as pd
import math
import numpy as np


```
Keywords:  
CP - Ordinary Conformal Prediction  
normCP/NCP - Normalized Conformal Prediction  
QR - Quantile Regression  
CQR - Conformalized Quantile Regressione
```

Load test prediction dataframe

In [4]:
# Ordinary CP
df = pd.read_pickle("/content/Calibration_df_run5")
df_test = pd.read_pickle("/content/Test_df_run5")

In [5]:
#NCPBLSTM
df_norm2 = pd.read_pickle("/content/Calibration_df_NORMCP_approach2") # /content/Calibration_df_normcp
df_test_norm2 = pd.read_pickle("/content/Test_df_NORMCP_approach2") # /content/Test_df_normcp

df['norm_log_left_error2'] = df_norm2['norm_log_left_error']
df['norm_log_right_error2'] = df_norm2['norm_log_right_error']
df_test['norm_log_left_error2'] = df_test_norm2['norm_log_left_error']
df_test['norm_log_right_error2'] = df_test_norm2['norm_log_right_error']

Ensemble technique: Ordinary CP + NormCP

Left edge

In [13]:
# 90% miscoverage rate
miscoverage = 0.1

In [14]:
# Ordinary CP
cp_left_conf = df['lpred_nonconformity']
cp_left_conf = cp_left_conf.sort_values(ascending=False)
significance_index = math.floor(miscoverage*(len(cp_left_conf) + 1))
cp_left_conf.iloc[significance_index]

0.06761092398009694

In [15]:
# normCP
cp_left_conf2 = df['lpred_nonconformity']/np.exp(-df['norm_log_left_error2'])
cp_left_conf2 = cp_left_conf2.sort_values(ascending=False)

significance_index2 = math.floor(miscoverage*(len(cp_left_conf2) + 1))
cp_left_conf2.iloc[significance_index2]

2.4338917899128134

In [16]:
check = 0
interval_len = []
ratio = []


for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']

  pred = df_test.iloc[i]['ledge_pred_sigma']

  left1 = (pred-cp_left_conf.iloc[significance_index])
  right1 = (pred+cp_left_conf.iloc[significance_index])
  interval1 = (right1-left1)

  delta = cp_left_conf2.iloc[significance_index2]*np.exp(-df_test['norm_log_left_error2']).iloc[i]
  left2 = (pred-delta)
  right2 = (pred+delta)
  interval2 = (right2-left2)

  if interval1 == min(interval1, interval2):
    right = right1
    left = left1
  elif interval2 == min(interval1, interval2):
    right = right2
    left = left2

  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.8847222222222222
ave interval: 0.11544422208059425
min interval: 0.0773372619244388
max interval: 0.1352218479601941
min ratio: 0.047641292748791524
max ratio: 0.48689692089832703


Right edge

In [17]:
# Ordinary CP
cp_right_conf = df['rpred_nonconformity']
cp_right_conf = cp_right_conf.sort_values(ascending=False)
significance_index = math.floor(miscoverage*(len(cp_right_conf) + 1))
cp_right_conf.iloc[significance_index]

0.09307463300756136

In [18]:
# normCP
cp_right_conf2 = df['rpred_nonconformity']/np.exp(-df['norm_log_right_error2'])
cp_right_conf2 = cp_right_conf2.sort_values(ascending=False)

significance_index2 = math.floor(miscoverage*(len(cp_right_conf2) + 1))
cp_right_conf2.iloc[significance_index2]

2.725528103996811

In [19]:
check = 0
interval_len = []
ratio = []


for i in range(len(df_test)):
  original = df_test.iloc[i]['i_rightline_sigma']

  pred = df_test.iloc[i]['redge_pred_sigma']

  left1 = (pred-cp_right_conf.iloc[significance_index])
  right1 = (pred+cp_right_conf.iloc[significance_index])
  interval1 = (right1-left1)

  delta = cp_right_conf2.iloc[significance_index2]*np.exp(-df_test['norm_log_right_error2']).iloc[i]
  left2 = (pred-delta)
  right2 = (pred+delta)
  interval2 = (right2-left2)

  if interval1 == min(interval1, interval2):
    right = right1
    left = left1
  elif interval2 == min(interval1, interval2):
    right = right2
    left = left2

  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.8682291666666667
ave interval: 0.1690340800413409
min interval: 0.13077163957686055
max interval: 0.18614926601512316
min ratio: 0.05575791282500983
max ratio: 0.6839189273084976
